In [2]:
import pandas as pd

# ▼ 入力CSVファイルのパス
input_csv_path = "gamei_apps_sales_summary_utf8bom_ver2.csv"

# ▼ 出力CSVファイルのパス
output_csv_path = "unique_titles.csv"

# ▼ CSVの読み込み
df = pd.read_csv(input_csv_path)

# ▼ タイトル列の重複を除いた一覧を取得
unique_titles = df["タイトル"].dropna().unique()

# ▼ データフレームに変換
titles_df = pd.DataFrame(unique_titles, columns=["タイトル"])

# ▼ 新しいCSVとして保存（UTF-8 BOM付きで文字化け防止）
titles_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

print(f"{len(titles_df)} 件のユニークなタイトルを {output_csv_path} に保存しました。")

393 件のユニークなタイトルを unique_titles.csv に保存しました。


In [4]:
import time
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ▼ ファイル設定
input_file = "unique_titles.csv"
output_file = "titles_with_release_dates_test.csv"

# ▼ Chromeオプション（必要に応じて headless）
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")

# ▼ ドライバ起動
driver = webdriver.Chrome(options=options)

# ▼ タイトル読み込み（3件だけ）
df = pd.read_csv(input_file).head(3).copy()
df["発売日"] = None

# ▼ メインループ
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/3] 検索中：{title}")

    try:
        # ① トップページから検索
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # ② 検索結果の描画を待ち、正しいリンク構造で取得
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        )
        links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]

        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            continue

        # ③ ゲーム紹介ページにアクセスして「発売日」を取得
        release_date = None
        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")
                for th in th_list:
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            release_date = td.text.strip()
                            break
                        except:
                            continue

                if release_date:
                    df.at[idx, "発売日"] = release_date
                    print(f" → 発売日：{release_date}")
                    break

            except Exception as sub_e:
                print(" → ページ処理中エラー:")
                traceback.print_exc()
                continue

        if not release_date:
            print(" → 発売日が見つかりません")

    except Exception as e:
        print(" → メイン処理エラー:")
        traceback.print_exc()
        continue

# ▼ ドライバ終了＆CSV保存
driver.quit()
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n✅ テスト完了！ → {output_file} に保存されました。")


[1/3] 検索中：スーパーガンダムロワイヤル
 → 発売日：2015/10/16
[2/3] 検索中：武器よさらば
 → ゲーム紹介ページが見つかりません
[3/3] 検索中：スーパーロボット大戦X-Ω
 → 発売日：2015/10/05

✅ テスト完了！ → titles_with_release_dates_test.csv に保存されました。


In [5]:
import time
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ▼ ファイル設定
input_file = "unique_titles.csv"
output_file = "titles_with_release_dates_test.csv"

# ▼ Chromeオプション（必要に応じて headless）
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")

# ▼ ドライバ起動
driver = webdriver.Chrome(options=options)

# ▼ タイトル読み込み（3件だけ）
df = pd.read_csv(input_file).head(3).copy()
df["発売日"] = None

# ▼ メインループ
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/3] 検索中：{title}")

    try:
        # ① トップページから検索
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # ② 検索結果を待つ（表示されているリンクを対象）
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
        )
        links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]

        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            continue

        # ③ ゲーム紹介ページにアクセスして「発売日」を取得
        release_date = None
        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")
                for th in th_list:
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            release_date = td.text.strip()
                            break
                        except:
                            continue

                if release_date:
                    df.at[idx, "発売日"] = release_date
                    print(f" → 発売日：{release_date}")
                    break

            except Exception as sub_e:
                print(" → ページ処理中エラー:")
                traceback.print_exc()
                continue

        if not release_date:
            print(" → 発売日が見つかりません")

    except Exception as e:
        print(" → メイン処理エラー:")
        traceback.print_exc()
        continue

# ▼ ドライバ終了＆CSV保存
driver.quit()
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n✅ テスト完了！ → {output_file} に保存されました。")


[1/3] 検索中：スーパーガンダムロワイヤル
 → 発売日：2015/10/16
[2/3] 検索中：武器よさらば
 → 発売日：2017/03/30
[3/3] 検索中：スーパーロボット大戦X-Ω
 → 発売日：2015/10/05

✅ テスト完了！ → titles_with_release_dates_test.csv に保存されました。
